# Práctica 2b

Desarrollar una aplicación que lleve a cabo distorsiones de la lente. Para ello los coeficientes de distorsión deben gobernarse a través de una interfaz.

In [2]:
import cv2
import numpy as np

def apply_distortion(image, k1, k2, p1, p2, k3):
    h, w = image.shape[:2]
    distCoeff = np.zeros((5,1), np.float64)
    distCoeff[0,0] = k1
    distCoeff[1,0] = k2
    distCoeff[2,0] = p1
    distCoeff[3,0] = p2
    distCoeff[4,0] = k3

    cam = np.eye(3, dtype=np.float32)
    cam[0,2] = w/2.0
    cam[1,2] = h/2.0
    cam[0,0] = 10.0
    cam[1,1] = 10.0

    distorted_img = cv2.undistort(image, cam, distCoeff)
    return distorted_img

image = cv2.imread('images/eii.png')
if image is None:
    raise FileNotFoundError("No se pudo cargar la imagen")

cv2.namedWindow("Distorsion interactiva")
cv2.createTrackbar("k1", "Distorsion interactiva", 50, 100, lambda x: None)
cv2.createTrackbar("k2", "Distorsion interactiva", 50, 100, lambda x: None)
cv2.createTrackbar("p1", "Distorsion interactiva", 50, 100, lambda x: None)
cv2.createTrackbar("p2", "Distorsion interactiva", 50, 100, lambda x: None)
cv2.createTrackbar("k3", "Distorsion interactiva", 50, 100, lambda x: None)

while True:
    k1 = (cv2.getTrackbarPos("k1", "Distorsion interactiva") - 50) / 5000.0
    k2 = (cv2.getTrackbarPos("k2", "Distorsion interactiva") - 50) / 5000.0
    p1 = (cv2.getTrackbarPos("p1", "Distorsion interactiva") - 50) / 5000.0
    p2 = (cv2.getTrackbarPos("p2", "Distorsion interactiva") - 50) / 5000.0
    k3 = (cv2.getTrackbarPos("k3", "Distorsion interactiva") - 50) / 5000.0

    distorted = apply_distortion(image, k1, k2, p1, p2, k3)

    cv2.imshow("Distorsion interactiva", distorted)
    key = cv2.waitKey(30) & 0xFF
    if key == ord("q") or key == 27:
        break

cv2.destroyAllWindows()


Funcionalidad de seleccionar el centro de distorsión con el ratón:

In [1]:
import cv2
import numpy as np

cx, cy = None, None

def apply_distortion(image, k1, k2, p1, p2, k3, center=None):
    h, w = image.shape[:2]
    distCoeff = np.zeros((5,1), np.float64)
    distCoeff[0,0] = k1
    distCoeff[1,0] = k2
    distCoeff[2,0] = p1
    distCoeff[3,0] = p2
    distCoeff[4,0] = k3

    cam = np.eye(3, dtype=np.float32)
    if center is None:
        cam[0,2] = w/2.0
        cam[1,2] = h/2.0
    else:
        cam[0,2] = center[0]
        cam[1,2] = center[1]

    cam[0,0] = 10.0
    cam[1,1] = 10.0

    distorted_img = cv2.undistort(image, cam, distCoeff)
    return distorted_img

def mouse_click(event, x, y, flags, param):
    global cx, cy
    if event == cv2.EVENT_LBUTTONDOWN:
        cx, cy = x, y

image = cv2.imread('images/eii.png')
if image is None:
    raise FileNotFoundError("No se pudo cargar la imagen")

cv2.namedWindow("Distorsion interactiva")
cv2.setMouseCallback("Distorsion interactiva", mouse_click)
cv2.createTrackbar("k1", "Distorsion interactiva", 50, 100, lambda x: None)
cv2.createTrackbar("k2", "Distorsion interactiva", 50, 100, lambda x: None)
cv2.createTrackbar("p1", "Distorsion interactiva", 50, 100, lambda x: None)
cv2.createTrackbar("p2", "Distorsion interactiva", 50, 100, lambda x: None)
cv2.createTrackbar("k3", "Distorsion interactiva", 50, 100, lambda x: None)

while True:
    k1 = (cv2.getTrackbarPos("k1", "Distorsion interactiva") - 50) / 5000.0
    k2 = (cv2.getTrackbarPos("k2", "Distorsion interactiva") - 50) / 5000.0
    p1 = (cv2.getTrackbarPos("p1", "Distorsion interactiva") - 50) / 5000.0
    p2 = (cv2.getTrackbarPos("p2", "Distorsion interactiva") - 50) / 5000.0
    k3 = (cv2.getTrackbarPos("k3", "Distorsion interactiva") - 50) / 5000.0

    distorted = apply_distortion(image, k1, k2, p1, p2, k3, center=(cx, cy) if cx is not None else None)

    display = distorted.copy()
    if cx is not None and cy is not None:
        cv2.circle(display, (cx, cy), 5, (0,0,255), -1)

    cv2.imshow("Distorsion interactiva", display)
    key = cv2.waitKey(30) & 0xFF
    if key == ord("q") or key == 27:
        break

cv2.destroyAllWindows()


error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2570: error: (-27:Null pointer) NULL window: 'Distorsion interactiva' in function 'cvGetTrackbarPos'
